# CEE 598: Uncertainty Quantification
## Assignment Two
### Noah Garfinkle (garfink2@illinois edu)
### 13 February 2020

# Imports and Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Problem 1

Using inverse transform sampling, write a code that can be used to generate n samples from the Cauchy distribution.  The Cauchy PDF and CDF are given by

$$
f_X\left(x\right)=\frac{1}{\pi\gamma\left[1+\left(\frac{x-x_0}{\gamma}\right)^2\right]}
$$

$$
F_X\left(x\right)=\frac{1}{\pi}\arctan{\left(\frac{x-x_0}{\gamma}\right)}+\frac{1}{2}
$$

Where $x_o$ is the median, and $\gamma$ is a scale factor.  The inputs to your code should be the number of samples $n$, and the numerical values for $x_0$ and $\gamma$.  Compute the generated samples with the target PDF, for a particular choice of $x_0$ and $\gamma$ (you’re free to choose any value).  You can use histogram plots or any available kernel density estimator codes.  Discuss your results.

In [2]:
x0 = 0
γ = 2

In [3]:
def pdf(x,x0=0,γ=2):
    f_denom = np.pi * γ * (1 + np.power((x-x0)/γ,2))
    f = 1.0 / f_denom
    return f

In [4]:
def cdf(x,x0=0,γ=2):
    F = (1/np.pi) * np.arctan((x-x0)/γ) + 1/2
    return F

In [5]:
def testPDF(x,x0=0,γ=2):
    f = stats.cauchy.pdf(x=x, loc=x0, scale=γ)
    return f

In [6]:
def testCDF(x,x0=0,γ=2):
    F = stats.cauchy.cdf(x=x, loc=x0, scale=γ)
    return F

Solve for the inverse of the CDF

$$
y=\frac{1}{\pi}\arctan{\left(\frac{x-x_0}{\gamma}\right)}+\frac{1}{2}
$$
$$
y-\frac{1}{2}=\frac{1}{\pi}\arctan{\left(\frac{x-x_0}{\gamma}\right)}
$$
$$
\pi y-\frac{1}{2}\pi=\arctan{\left(\frac{x-x_0}{\gamma}\right)}
$$
$$
\tan{\left(y\pi-\frac{1}{2}\pi\right)}=\frac{x-x_0}{\gamma}
$$
$$
\gamma tan{\left(y\pi-\frac{1}{2}\pi\right)}=x-x_0
$$
$$
x=\gamma tan{\left(y\pi-\frac{1}{2}\pi\right)}+x_0\rightarrow Inverse\ CDF
$$

In [7]:
def inverseCDF(P,x0=0.0,γ=2.0):
    x = γ*np.tan(P*np.pi-(1/2)*np.pi) + x0
    return x

In [8]:
def cauchy(n,x0=0,γ=2):
    inverseCDFXs = []
    for i in range(0,n):
        u = np.random.uniform()
        inverseCDFXs.append(inverseCDF(u,x0,γ))
    return inverseCDFXs

In [9]:
def plotController(x0=0.0,γ=2.0,n=10000):
    xs = np.linspace(0,10,100)
    fs = [pdf(x,x0,γ) for x in xs]
    testfs = [testPDF(x,x0,γ) for x in xs]

    Fs = [cdf(x,x0,γ) for x in xs]
    testFs = [testCDF(x,x0,γ) for x in xs]

    fig,ax = plt.subplots(2,2,figsize=(20,20))
    ax[0,0].plot(xs,fs,label="My PDF", alpha=0.5)
    ax[0,0].fill_between(xs,testfs,hatch='x',facecolor="white",label="Numpy PDF")

    ax[0,1].plot(xs,Fs,label="My CDF", alpha=0.5)
    ax[0,1].fill_between(xs,testFs,hatch='+',facecolor="white",label="Numpy CDF",alpha=0.5)

    ax[0,0].set_title(f"PDF Test: x0:{x0}, γ={γ}")
    ax[0,1].set_title(f"CDF Test: x0:{x0}, γ={γ}")

    ax[0,0].set_xlabel("x")
    ax[0,0].set_ylabel("P")

    ax[0,1].set_xlabel("x")
    ax[0,1].set_ylabel("P")

    ax[0,0].legend()
    ax[0,1].legend()
    
    Ps = np.linspace(0.01,0.99,100)
    inverseXs = [inverseCDF(P,x0,γ) for P in Ps]
    ax[1,0].plot(inverseXs,Ps,label="Inverse CDF")
    ax[1,0].legend()
    ax[1,0].set_title(f"Inverse CDF: x0:{x0}, γ={γ}")
    
    c = cauchy(n,x0=x0,γ=γ)
    testC = stats.cauchy.rvs(size=n, loc=x0, scale=γ)
    
    # compare two distributions
    # https://stats.stackexchange.com/questions/354035/how-to-compare-the-data-distribution-of-2-datasets
    ksTest = stats.ks_2samp(c,testC)
    
    sns.kdeplot(c,ax=ax[1,1],label="My Generator",alpha=0.5)
    sns.kdeplot(testC,ax=ax[1,1],label="Numpy Generator",alpha=0.5)
    ax[1,1].legend()
    ax[1,1].set_title(f"Comparison of Implemented and Numpy Cauchy Random Number Generator\nx0:{x0}, γ={γ}\nKolmogorov-Smirnov Test: Statistic-{ksTest.statistic:,.3f},pValue-{ksTest.pvalue:,.3}\nμ={np.mean(c)}")

In [11]:
interact(plotController,x0=widgets.FloatSlider(min=-10,max=10,step=0.5,value=0),γ=widgets.FloatSlider(min=1,max=10,step=0.1,value=1),n=widgets.IntSlider(min=10000,max=1000000,step=10000,value=100000))

interactive(children=(FloatSlider(value=0.0, description='x0', max=10.0, min=-10.0, step=0.5), FloatSlider(val…

<function __main__.plotController(x0=0.0, γ=2.0, n=10000)>

# Problem 2

You are asked to provide a random number generator from the Gamma PDF, $Gamma(x;2,2)$, given by

$$
f_X\left(x\right)=\frac{1}{4}xe^{-\frac{x}{2}}
$$

You are supposed to use rejection sampling, and choose the envelope PDF, i.e. $cg(x)$, to be a scaled version of Cauchy PDF (given in Problem 1).

In [ ]:
def gammaPDF(x):
    f = 1/4 * x * np.exp(-x/2)
    return f

## Problem 2.1

Show how you modify your Cauchy PDF to become an upper bound for the Gamma PDF everywhere.  The parameters that you can vary so that this upper bound condition is satisfied are $x_0$, $\gamma$, and also $c$.  There are several combinations of these parameters that can make the Cauchy PDF an envelope distribution.  Discussion how some of these conditions can be more favorable than others.

## Problem 2.2

Write an acceptance-rejection sampling code that can be used to generate $n$ samples from the given Gamma PDF.  Use the code you wrote for Problem 1 to draw samples from $g_X(x)$ within your rejection sampling algorithm.


## Problem 2.3

Compare the generated samples with the targeted PDF.  You can use histogram plots or available kernel density estimator codes.  Discuss your results.
